# ESERCIZIO 5: MERGE E JOIN 03.07
* * *

### OBBIETTIVO: classificare gli Stati e territori americani secondo la densità di popolazione del 2010.

+ **Importo le librerie necessarie**

In [1]:
import numpy as np
import pandas as pd

+ **Scarico i tre dataframe da GitHub e li visualizzo con display e la funzione head**

In [2]:
pop = pd.read_csv('C:/Users/Utente/Desktop/state-population.csv')
areas = pd.read_csv('C:/Users/Utente/Desktop/state-areas.csv')
abbrevs = pd.read_csv('C:/Users/Utente/Desktop/state-abbrevs.csv')

display(pop.head(), areas.head(), abbrevs.head())

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


**Unisco il dataframe pop e abbrev:**
+ **tenendo anche le colonne non comuni** how='outer'
+ **(siccome non ci sono colonne comuni) mettendo a sx state/region e a dx abbreviation** left_on='state/region', right_on='abbreviation'
+ **per evitare duplicati uso DROP quindi elimino l'ultima colonna abbreviation** merged.drop('abbreviation', 1)

In [3]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')

merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


+ **Controllo se ci sono valori nulli sulle righe** Si ce ne sono (True per population e State)

In [4]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

+ **Iniziando con population visualizzo quali sono i valori nulli** Sembrano tutti provenineti da Puerto Rico, questa mancanza di valori non può essere risolta o sopperita: questi dati probabilmente mancano/non sono stati rilevati o comunicati.

In [5]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


+ **Poi analizzo i valori nulli di state** Sono inerenti a Puerto Rico e agli Stati Uniti, il problema è risolvibile

In [6]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

+ **Si può velocemente risolvere il problema per state, abbinando a PR Puerto Rico e a USA United States**

In [7]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

+ **Possiamo unire il terzo dataframe, area, specificando la chiave 'state' e considerando i NaN (any, qualsiasi) a sx**

In [8]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


+ **Controllo se ci sono valori mancanti NaN nel dataframe finale** Appaiono dei dati mancanti in area.

In [9]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

+ **Analizziamo in quali stati/regioni si trovasno questi valori** Negli Stati Uniti nel loro complesso

In [10]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

+ **Ai fini del nostro obbiettivo, possiamo quindi eliminare questi dati in quanto non ci interessa il valore nel complesso, bensì delle singole regioni/stati** Utilizzo dropna per eliminarli

In [11]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


+ **A questo punto avendo il dataframe seleziono l'anno 2010 e la popolazione totale (non suddivisa in fasce di età)** Utilizzo la funzione query

In [12]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


+ **Calcoliamo la densità di popolazione e la ordiniamo in ordine decrescente**

In [13]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [14]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

+ **Visualizzo la fine della lista, quindi i valori più bassi di densità di popolazione** 

In [15]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64